# NBA Data Analysis for Prediction Model

## Table of Contents <a id="table_of_contents"></a>

0. [Import Libraries](#imports)
1. [Import Data](#import_data)
2. Data Exploration
3. Data Cleaning
4. Data Preparation
5. Benchmark Model
6. Visualization ?

## 0. Importing Libraries <a id="imports"></a>

[Back to top](#table_of_contents)

In [1]:
!pip install opendatasets

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/geometry-0.0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
pip install pypyodbc

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.12/site-packages/geometry-0.0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import matplotlib
from matplotlib import pyplot as plt
import opendatasets as od
import os

## 1. Import Data <a id="import_data"></a>

[Back to top](#table_of_contents)

In [4]:
#create connection to database file in Kaggle
dataset = "https://www.kaggle.com/datasets/wyattowalsh/basketball"
od.download(dataset)

Skipping, found downloaded files in "./basketball" (use force=True to force download)


In [5]:
# 1.Connect to SQL database
connection = sql.connect('basketball/nba.sqlite')
# Test SQL connectivity to database by writing a SQL query
test_query = "SELECT * FROM game"
# Load the data into a pandas DataFrame
df = pd.read_sql_query(test_query, connection)
# Display the DataFrame
print(df)
# Close Connection if necessary
connection.close()

      season_id team_id_home team_abbreviation_home           team_name_home  \
0         21946   1610610035                    HUS          Toronto Huskies   
1         21946   1610610034                    BOM        St. Louis Bombers   
2         21946   1610610032                    PRO  Providence Steamrollers   
3         21946   1610610025                    CHS            Chicago Stags   
4         21946   1610610028                    DEF          Detroit Falcons   
...         ...          ...                    ...                      ...   
65693     42022   1610612748                    MIA               Miami Heat   
65694     42022   1610612748                    MIA               Miami Heat   
65695     42022   1610612743                    DEN           Denver Nuggets   
65696     32022   1610616834                    LBN              Team LeBron   
65697     32022   1610616834                    LBN              Team LeBron   

          game_id            game_date 

## 2. Data Exploration

[Click here](https://www.nba.com/stats/help/glossary) for an NBA stat glossary.
[Click here](https://en.wikipedia.org/wiki/Basketball_statistics) for general basketball statistics acronyms.

Slug for CSV folder files (for the purpose of viewing the tables here): /basketball/csv/[table name].csv

* fgm = field goals made
* ftm = free throws made
* pts = points
* reb = rebounds
* oreb = offensive rebounds
* dreb = defensive rebounds
* ast = assists
* stl = steals
* blk = blocks
* tov = turnovers
* td = triple double
* wl = win/lose
* ft_pct_away = free throws % of away team

NBA's efficiency rating: (PTS + REB + AST + STL + BLK − ((FGA − FGM) + (FTA − FTM) + TO))

## 3. Data Cleaning

In [8]:
# 1.Connect to SQL database
explore_connection = sql.connect('basketball/nba.sqlite')

cursor = explore_connection.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS game_cleaned AS
    SELECT team_id_home, team_abbreviation_home, team_name_home, game_id, wl_home, fga_home, fg_pct_home, ftm_home, pts_home, fta_home, fgm_home, oreb_home, dreb_home, reb_home, ast_home, stl_home, blk_home, tov_home, pf_home, team_id_away, team_name_away, wl_away, fg_pct_away, ft_pct_away, pts_away, fgm_away, oreb_away, dreb_away, reb_away, ast_away, stl_away, blk_away, tov_away, pf_away
    FROM game
    WHERE fga_home IS NOT NULL AND fta_home IS NOT NULL AND ftm_home IS NOT NULL AND oreb_home IS NOT NULL AND dreb_home IS NOT NULL AND stl_home IS NOT NULL AND ast_home IS NOT NULL AND reb_home IS NOT NULL AND blk_home IS NOT NULL AND tov_home IS NOT NULL AND pf_home IS NOT NULL AND pts_home IS NOT NULL AND team_id_away IS NOT NULL AND team_name_away IS NOT NULL AND wl_away IS NOT NULL AND fg_pct_away IS NOT NULL AND ft_pct_away IS NOT NULL AND pts_away IS NOT NULL AND fgm_away IS NOT NULL AND oreb_away IS NOT NULL AND dreb_away IS NOT NULL AND reb_away IS NOT NULL AND ast_away IS NOT NULL AND stl_away IS NOT NULL AND blk_away IS NOT NULL AND tov_away IS NOT NULL AND pf_away IS NOT NULL AND pts_away IS NOT NULL;
''')
connection.close()

In [11]:
# 1.Connect to SQL database
add_calculation_connection = sql.connect('basketball/nba.sqlite')
cursor.execute('''
    ALTER TABLE game_cleaned ADD COLUMN eff_home;
''')

cursor.execute('''
    ALTER TABLE game_cleaned ADD COLUMN eff_away;
''')

cursor.execute('''
    UPDATE game_cleaned
    SET eff_home = (pts_home + reb_home + ast_home + stl_home + blk_home - ((fga_home - fgm_home) + (fta_home - ftm_home) + tov_home)),
        eff_away = (pts_away + reb_away + ast_away + stl_away + blk_away - ((fga_away - fgm_away) + (fta_away - ftm_away) + tov_away))
''')

display_game_cleaned_query = "SELECT * FROM game_cleaned"
game_cleaned_df = pd.read_sql_query(display_game_cleaned_query, explore_connection)
print(game_cleaned_df)
connection.close()

OperationalError: no such column: fga_home